In [ ]:
#!pip3 install torch torchvision torchaudio

In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.3163, 0.7106, 0.4187],
        [0.1085, 0.2414, 0.2090],
        [0.1190, 0.7055, 0.2739],
        [0.6191, 0.7528, 0.8166],
        [0.7568, 0.0670, 0.7406]])


In [2]:
torch.cuda.is_available()

True

Awesome